<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcoteran/deeplearning/blob/master/notebooks/2.4_machinelearning_dimensionalityreduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marcoteran/deeplearning/blob/master/notebooks/2.4_machinelearning_dimensionalityreduction.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Abrir en Kaggle" title="Abrir y ejecutar en Kaggle"/></a>
  </td>
</table>

### Ejemplo de código
# Sesión 06: Reducción de la dimensionalidad y aprendizaje de la representación
## Deep Learning y series de tiempo

**Name:** Marco Teran **E-mail:** marco.tulio.teran@gmail.com,
[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

#### Importar librerías importantes

Definimos primero unas librerías y funciones que vamos a usar a durante la sesión:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#!rm -rf data/
#!rm -rf data.z*
#!mkdir -p data/
#!wget -O data.zip https://github.com/marcoteran/deeplearning/raw/master/notebooks/data.zip
#!unzip data.zip
#!ls

# Análisis de Componentes Principales (PCA)

El **Análisis de Componentes Principales (PCA)** es una técnica de reducción de dimensionalidad lineal que se utiliza para extraer información de un *espacio de alta dimensionalidad* al proyectarlo en un subespacio de menor dimensión.
* PCA intenta preservar las partes esenciales que tienen mayor variación en los datos y eliminar las partes no esenciales con menos variación.
* Las dimensiones son características que representan los datos
    - Ejemplo: en una imagen de 28 X 28 píxeles hay 784 elementos de imagen que son las dimensiones o características que juntas representan esa imagen.
* PCA es una técnica de reducción de dimensionalidad no supervisada, lo que significa que se pueden agrupar puntos de datos similares basados en la correlación entre características sin ninguna supervisión o etiquetas.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/examplepca.png" width="30%">

PCA es un procedimiento estadístico que utiliza una transformación ortogonal para convertir un conjunto de observaciones de variables posiblemente correlacionadas en un conjunto de valores de variables linealmente no correlacionadas llamadas componentes principales.

**Nota:** Características, Dimensiones y Variables se refieren a lo mismo. Se utilizan indistintamente.

### Aplicaciones del PCA
* **La Visualización de Datos:** El problema en el mundo actual es la gran cantidad de datos y las variables/características que definen esos datos. Para resolver un problema en el que los datos son clave, se necesita una exploración exhaustiva de los datos como encontrar cómo están correlacionadas las variables o entender la distribución de algunas variables. La visualización puede ser un desafío y casi imposible debido a la gran cantidad de variables o dimensiones en las que se distribuyen los datos. PCA puede proyectar los datos en una dimensión más baja, permitiéndote visualizar los datos en un espacio 2D o 3D con el ojo humano.
* **Aceleración de un Algoritmo de Machine Learning (ML):** Dado que la idea principal de PCA es la reducción de la dimensionalidad, se puede aprovechar para acelerar el tiempo de entrenamiento y prueba de un algoritmo de ML si los datos tienen muchas características y el aprendizaje del algoritmo de ML es demasiado lento. A nivel abstracto, se toma un conjunto de datos con muchas características y se simplifica seleccionando algunos componentes principales de las características originales.

Consulta [A Tutorial on Principal Component Analysis](https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf) para una decripción intuitiva y detallada de PCA y SVD

## Intuición detrás del PCA

Tenemos los siguientes datos 2D y nos gustaría encontrar una proyección en 1D que preserve la máxima cantidad de variabilidad.


In [ ]:
np.random.seed(1)
X = np.dot(np.random.random(size=(2, 2)), np.random.normal(size=(2, 200))).T+10

# Centra los datos en 0,0
X=X-np.mean(X, axis=0)

plt.scatter(X[:,0], X[:,1])

La proyección de un vector en otro es la proyección ortogonal de un vector sobre otro en un espacio vectorial, que mide la magnitud de la componente de un vector en la dirección de otro vector.

Recuerda que la proyección de un vector $\vec{x}$ en otro vector $\vec{v}$ (consulta [aquí](https://matthew-brett.github.io/teaching/vector_projection.html)) viene dada por:

$$c = \frac{\vec{v}\times \vec{x}}{||\vec{v}||^2}$$


$$proj_\vec{v} \vec{x} = \vec{v} c$$


donde $c$ es el tamaño de la proyección de  $\vec{x}$ sobre $\vec{v}$

### Inspeccionamos algunas proyecciones

El siguiente código de proyección de vectores representa la proyección de un conjunto de datos (X) en una dirección aleatoria, usando la técnica de proyección vectorial.
* Se generan tres gráficos que muestran la proyección de los datos en una dirección aleatoria.
* Para cada dirección, se calcula el factor de escala de proyección (c) para ajustar la magnitud de la proyección.
* Los puntos proyectados se muestran en rojo, mientras que los datos originales se muestran en azul.
* La línea negra representa el vector de proyección y la desviación estándar de los factores de escala se muestra en el título.

In [ ]:
plt.figure(figsize=(15,3))

unit_vector = lambda angle: np.array([np.cos(angle), np.sin(angle)])

for i in range(3):
    plt.subplot(1,3,i+1)
    angle = np.random.random()*np.pi*2
    v = unit_vector(angle)

    c = X.dot(v.reshape(-1,1))/(np.linalg.norm(v)**2)
    Xp = np.repeat(v.reshape(-1,2),len(X),axis=0)*c

    plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="original data")
    plt.scatter(Xp[:,0], Xp[:,1], color="red", alpha=.5, label="projected data")
    plt.axvline(0, color="gray")
    plt.axhline(0, color="gray")
    plt.plot([0,v[0]], [0,v[1]], color="black", lw=3, label="projection vector")
    plt.axis('equal')
    plt.ylim(-2,2)
    plt.title("$\\alpha$=%.2f rads, proj std=%.3f"%(angle, np.std(c)))
    if i==2:
        plt.legend(loc="center left", bbox_to_anchor=(1.01,.5))

### Encontremos las proyecciones con mayor y menor std por fuerza bruta

Se busca obtener las proyecciones de un conjunto de datos en diferentes ángulos para encontrar la dirección con la máxima y mínima variabilidad en los datos. Se muestra gráficamente cómo varía la desviación estándar de las proyecciones a medida que se rota el vector de proyección.
* El código calcula la desviación estándar de las proyecciones de un conjunto de datos en diferentes ángulos.
* Utiliza esta información para encontrar la dirección con la máxima y mínima variabilidad en los datos.
* Muestra gráficamente cómo varía la desviación estándar de las proyecciones a medida que se rota el vector de proyección.

In [ ]:
def get_maxmin_projections(X):
    stds = []
    angles = np.linspace(0,np.pi*2, 100)
    for a in angles:
        v = np.array([np.cos(a), np.sin(a)])
        c = X.dot(v.reshape(-1,1))/(np.linalg.norm(v)**2)
        stds.append(np.std(c))
    v2 = unit_vector(angles[np.argmin(stds)])
    v1 = unit_vector(angles[np.argmax(stds)])
    
    return angles, stds, v1, v2
angles, stds, v1, v2 = get_maxmin_projections(X)

plt.plot(angles, stds)
plt.xlabel("projection $\\alpha$ (in rads)")
plt.ylabel("projection std")

In [ ]:
plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="original data")
plt.axvline(0, color="gray")
plt.axhline(0, color="gray")
plt.plot([0,v1[0]], [0,v1[1]], color="black", lw=5, label="max std projection vector")
plt.plot([0,v2[0]], [0,v2[1]], color="black", ls="--", lw=2, label="min std projection vector")
plt.axis('equal')
plt.ylim(-2,2)
plt.legend(loc="center left", bbox_to_anchor=(1.01,.5))

## Implementación de PCA en Scikit-Learn

PCA en Scikit-Learn se utiliza principalmente para reducir la dimensionalidad de los datos y permitir una visualización más fácil de los datos, pero también puede ayudar a reducir el ruido y mejorar la precisión del modelo de Machine Learning.
* PCA en Scikit-Learn se puede utilizar en una variedad de aplicaciones, como la reducción de dimensionalidad de datos, la visualización de datos y la eliminación de características irrelevantes o redundantes.
* Scikit-Learn ofrece una implementación eficiente de PCA, que se puede ajustar a diferentes tipos de datos y configuraciones de parámetros.
* La función de PCA en Scikit-Learn también proporciona una variedad de opciones para personalizar la salida y la interpretación de los resultados de PCA.

* **Estos son los componentes principales**
* **Observa que su dimensionalidad es la misma que los datos originales**

Esto es lo que PCA nos da:

In [ ]:
# Importamos el paquete PCA de scikit-learn
from sklearn.decomposition import PCA

# Creamos una instancia de PCA con dos componentes
pca = PCA(n_components=2)
# Ajustamos la instancia de PCA con los datos X
pca.fit(X)

# Imprimimos los componentes calculados por scikit-learn
print("sklearn PCA components")
print(pca.components_)

# Imprimimos los componentes calculados por fuerza bruta
print("brute force components")
print(v1)
print(v2)

Pero de modo mucho más eficiente

In [ ]:
%timeit pca.fit(X)

In [ ]:
%timeit get_maxmin_projections(X)

### Podemos usar el componente mayor para reducir la dimensionalidad de nuestros datos de 2D a 1D

A continuación se muestra cómo podemos reducir la dimensionalidad de nuestros datos de 2D a 1D utilizando el componente mayor.
* La ecuación que se muestra a continuación representa la transformación que se realiza en los datos originales para obtener los datos transformados:

$$\mathbf{X_t} = \mathbf{X} \times \mathbf{V}$$

Donde $\mathbf{X}$ representa nuestros datos originales, $\mathbf{V}$ es el vector de componentes seleccionados y $\mathbf{X_t}$ son los datos transformados.

Es importante mencionar que este tipo de transformaciones son lineales, es decir, que se realizan mediante rotaciones y escalados de los datos originales.
* Esta restricción nos permite utilizar técnicas de álgebra lineal para encontrar las transformaciones óptimas que mejor representen nuestros datos (rotaciones y escalado)

In [ ]:
# Creamos un objeto PCA con 1 componente para reducir la dimensionalidad de los datos de X a 1D
pca = PCA(n_components=1)

# Ajustamos el modelo PCA a los datos X
pca.fit(X)

# Transformamos los datos X al espacio reducido con PCA
Xt = pca.transform(X)[:,0]

# Graficamos los datos originales X en azul y los datos transformados Xt en rojo en el eje horizontal
# El eje vertical está fijado en 0 para visualizar mejor la reducción de dimensionalidad
plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="$\mathbf{X}$: original data")
plt.scatter(Xt, [0]*len(Xt), color="red", alpha=.5, label="$\mathbf{X_t}$: reduced data")

# Establecemos el mismo rango en ambos ejes para una mejor visualización y agregamos una leyenda
plt.axis("equal");
plt.legend(loc="center left", bbox_to_anchor=(1.01,.5))

Y podemos también recontruir los datos 2D después de la transformación

In [ ]:
# Obtenemos el primer componente principal
v0 = pca.components_[0]

# Proyectamos nuestros datos sobre el primer componente
c = X.dot(v0)

# Obtenemos la reconstrucción de los datos proyectados usando únicamente el primer componente
Xr = np.r_[[i*v0 for i in c]]

# Graficamos los datos originales y los datos reconstruidos
plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="original data")
plt.scatter(Xr[:,0], Xr[:,1], color="red", alpha=.5, label="reconstructed data from largest component")
# Agregamos una leyenda y definimos el tamaño de la figura
plt.legend(loc="center left", bbox_to_anchor=(1.01,.5))
plt.figure(figsize=(8,6))

## Reducción de dimensionalidad para tareas de clasificación

La reducción de dimensionalidad PCA puede mejorar la eficiencia computacional de los modelos de clasificación al reducir el número de características.
* La eliminación de características redundantes o irrelevantes mediante PCA puede mejorar la precisión y generalización del modelo.
* PCA puede ayudar a visualizar mejor los datos y las relaciones entre las características, lo que puede facilitar la interpretación y comprensión del modelo y de los datos.

In [ ]:
#Cargando el dataset mnist1.5k con pandas
mnist = pd.read_csv("data/mnist1.5k.csv.gz", compression="gzip", header=None).values

# Seleccionando las columnas de imágenes del dataset
d = mnist[:, 1:785]

# Seleccionando la columna de las clases del dataset
c = mnist[:, 0]

# Imprimiendo la dimensión de las imágenes y las clases
print("Dimensión de las imágenes y las clases: ", d.shape, c.shape)

El código selecciona aleatoriamente 50 imágenes y sus correspondientes etiquetas de un conjunto de datos de imágenes y las muestra en una figura con sus etiquetas correspondientes.

In [ ]:
perm = np.random.permutation(range(d.shape[0]))[0:50]
random_imgs   = d[perm]
random_labels = c[perm] 
fig = plt.figure(figsize=(10,6))
for i in range(random_imgs.shape[0]):
    ax=fig.add_subplot(5,10,i+1)
    plt.imshow(random_imgs[i].reshape(28,28), interpolation="nearest", cmap = plt.cm.Greys_r)
    ax.set_title(int(random_labels[i]))
    ax.set_xticklabels([])
    ax.set_yticklabels([])

### Analisis de componentes principales

El siguiente código carga un conjunto de datos de imágenes MNIST y realiza reducción de dimensionalidad mediante el análisis de componentes principales (PCA). Luego, muestra algunas de las componentes principales (PCs) obtenidas y también visualiza algunas imágenes originales y sus correspondientes reconstrucciones a partir de las PCs. Finalmente, utiliza un clasificador Naive Bayes y valida su desempeño en los datos originales y los datos transformados por PCA.

In [ ]:
# Lee el conjunto de datos de imágenes MNIST y separa las características (pixeles) de las etiquetas (números escritos a mano).
mnist = pd.read_csv("data/mnist1.5k.csv.gz", compression="gzip", header=None).values
X=mnist[:,1:785]
y=mnist[:,0]

In [ ]:
# Crea una instancia de PCA con 60 componentes y ajusta los datos de entrenamiento.
pca = PCA(n_components=60)
Xp = pca.fit_transform(X)

#### Obtenemos los componentes principales
Visualiza 60 componentes principales como imágenes de 28x28 píxeles.

In [ ]:
cols=20
plt.figure(figsize=(15,3))
for i in range(len(pca.components_)):
    plt.subplot(np.ceil(len(pca.components_)/15.),15,i+1)
    plt.imshow((pca.components_[i].reshape(28,28)), cmap = plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

#### Verificamos la reconstrucción con los componentes principales
Visualiza 6 imágenes originales y sus reconstrucciones a partir de las componentes principales.

In [ ]:
plt.figure(figsize=(10,6))
for i in range(6):
    plt.subplot(3,6,i+1)
    k = np.random.randint(len(X))
    plt.imshow((np.sum((pca.components_*Xp[k].reshape(-1,1)), axis=0)).reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])
    plt.subplot(3,6,6+i+1)
    plt.imshow(X[k].reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

### Clasificación en el nuevo espacio de representación

Utiliza el clasificador Naive Bayes y calcula el desempeño de la clasificación en los datos originales y los datos transformados por PCA.
* El **clasificador de Naive Bayes** es un algoritmo de aprendizaje supervisado que se basa en el teorema de Bayes para predecir la clase de un dato nuevo a partir de sus características. El teorema de Bayes es un principio matemático que describe cómo actualizar la probabilidad de una hipótesis a medida que se obtiene nueva evidencia. Algunas características importantes de este clasificador son:
    - Es un método simple y rápido que puede ser utilizado en conjuntos de datos grandes.
    - Es especialmente útil cuando se tienen muchas características y se desea evitar el problema de la maldición de la dimensionalidad.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

print(np.mean(cross_val_score(GaussianNB(), X, y, cv=5)))
print(np.mean(cross_val_score(GaussianNB(), Xp, y, cv=5)))

El segundo modelo, el que utilizó PCA, tiene un mejor rendimiento en términos de precisión que el primero que no utilizó PCA.

#### Observa la nueva representación de la primera imagen
Muestra la primera instancia de los datos transformados.

In [ ]:
Xp[0]

### Usando pipelines

Debemos de tener cuidado cuando usamos transformaciones en clasificación, ya que tenemos que ajustarlas (de manera no supervisada) sólo con los datos de entrenamiento:

In [ ]:
# Importamos la clase Pipeline del módulo sklearn.pipeline
from sklearn.pipeline import Pipeline

#Creamos un pipeline que primero hace reducción de dimensionalidad PCA y luego aplica el clasificador Naive Bayes
pip = Pipeline([("PCA", PCA(n_components=60)), ("gaussian", GaussianNB())])

# Obtenemos el resultado del score de validación cruzada utilizando el pipeline en los datos originales X e y
print(np.mean(cross_val_score(pip, X,y, cv=5 )))

## Singular Value Decomposition

Singular Value Decomposition (SVD) es una técnica matemática que descompone una matriz de datos en tres matrices más simples y significativas que pueden utilizarse para reducir la dimensionalidad de los datos, extraer patrones y características, y hacer la recomposición de los datos originales.

Características de SVD:
* Puede ser utilizado para reducción de dimensionalidad, detección de patrones y características, y análisis de correlación.
* Es una técnica no supervisada que no requiere de etiquetas o clases de datos para operar.
* Es ampliamente utilizado en aplicaciones como procesamiento de señales, reconocimiento de voz, minería de datos y aprendizaje automático.

En la fórmula presentada, $\mathbf{X}$ representa la matriz de datos original que se desea descomponer en tres matrices más simples. $\mathbf{U}$ y $\mathbf{V}^*$ representan matrices unitarias que contienen información sobre las direcciones y magnitudes de las variaciones de los datos en cada dimensión. 

$$\mathbf{X} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^*$$ 

donde:

- $\mathbf{X}$ son nuestros datos
- $\mathbf{U}$ es unitaria (sus columnas y filas son ortonormales, forman una base)
- $\mathbf{V}^*$ es unitaria (sus columnas y filas son ortonormales, forman una base)


$\mathbf{\Sigma}$ es una matriz diagonal que contiene los valores singulares, que representan la importancia relativa de cada dimensión.
La matriz $\mathbf{X}$ puede ser recompuesta como el producto de las tres matrices: $\mathbf{U}$, $\mathbf{\Sigma}$ y $\mathbf{V}^*$, lo que permite una representación más simple y eficiente de los datos.


In [ ]:
# Calcular la descomposición en valores singulares de los datos de entrada X
U, s, V = np.linalg.svd(X)

In [ ]:
# Obtener la forma de las matrices U, s y V
# U es una matriz unitaria cuyas columnas y filas son ortonormales (forman una base)
# s es un vector que contiene los valores singulares de X
# V es una matriz unitaria cuyas columnas y filas son ortonormales (forman una base)
U.shape, s.shape, V.shape

Reconstruimos la matriz diagonal s

In [ ]:
# Convertir el vector de valores singulares en una matriz diagonal
s = np.diag(s)
# Añadir filas de ceros a la matriz de valores singulares para que tenga la misma forma que U
s = np.vstack([s, np.zeros((U.shape[0]-s.shape[0], s.shape[1]))])

Verificamos las propiedades SVD

In [ ]:
print("verificando si U es unitaria: ", np.allclose(U.dot(U.T), np.eye(U.shape[0])))
print("verificando si las filas de U son unitarias: ", np.allclose(np.linalg.norm(U, axis=1), np.ones(U.shape[0])))
print("verificando si las columnas de U son unitarias: ", np.allclose(np.linalg.norm(U, axis=0), np.ones(U.shape[1])))
print("verificando si V es unitaria: ", np.allclose(V.T.dot(V), np.eye(V.shape[0])))
print("verificando si las filas de V son unitarias: ", np.allclose(np.linalg.norm(V, axis=1), np.ones(V.shape[0])))
print("verificando si las columnas de V son unitarias: ", np.allclose(np.linalg.norm(V, axis=0), np.ones(V.shape[1])))
print("verificando la reconstrucción de X: ", np.allclose(U.dot(s).dot(V), X))

$\Sigma$ viene ordenada, y cada coeficiente cuantifica cuanto contribuye cada base en $V$ a la variabilidad de los datos originales.

In [ ]:
plt.plot(np.diagonal(s))
plt.xlabel("component index of $\Sigma$");
plt.ylabel("component value")

### Observa que los componentes de PCA y $V^*$ de SVD son los mismos

Aunque a veces vengan con signo distinto

In [ ]:
Xm = X-np.mean(X, axis=0)
U,s,V = np.linalg.svd(Xm)
pca = PCA(n_components=60)
pca.fit(Xm);

In [ ]:
i = np.random.randint(pca.n_components)
plt.figure(figsize=(12,3))
plt.subplot(121)
plt.plot(pca.components_[i])
plt.title("PCA component %d"%i)
plt.subplot(122)
plt.title("SVD $V^*$ row %d"%i)
plt.plot(V[i])

Por ejemplo si queremos preservar solamente los vectores base de SVD que contienen el 40% de la variabilidad

In [ ]:
n_components = np.argwhere(np.cumsum(s)/np.sum(s)>.4)[0][0]
print("Keeping %d components"%n_components)

In [ ]:
c = V[:n_components]

In [ ]:
cols=20
plt.figure(figsize=(15,3))
for i in range(len(c)):
    plt.subplot(np.ceil(len(c)/15.),15,i+1)
    plt.imshow((c[i].reshape(28,28)), cmap = plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

Lo cual corresponde a los mismos componentes mencionados arriba para PCA.

Los componentes de PCA y las columnas de la matriz $V^*$ de SVD son los mismos porque el proceso de SVD es esencialmente un proceso de PCA. Al aplicar SVD a una matriz de datos, la matriz de $V^*$ devuelve las componentes principales de la matriz original. Por lo tanto, los vectores de la matriz de $V^*$ se pueden utilizar como componentes principales para la reducción de la dimensionalidad, de la misma manera que los componentes principales se obtienen directamente del proceso de PCA.

Ambos métodos realizan una descomposición de los datos para identificar las componentes principales y reducir la dimensionalidad, por lo que los componentes resultantes son los mismos.

## Non negative matrix factorization

Non-negative matrix factorization (NMF) es un método de factorización de matrices que descompone una matriz no negativa en dos matrices no negativas de menor rango.

Características de NMF:
* Se utiliza para reducción de dimensionalidad, extracción de características y clustering.
* Las matrices resultantes son interpretables y pueden ser utilizadas para identificar patrones y relaciones entre los datos.
* Es especialmente útil para datos en los que se espera que los valores sean no negativos, como en imágenes o señales.

El objetivo de NMF es descomponer una matriz no negativa $V \in \mathbb{R}+^{m\times n}$ en el producto $W \times H$, donde $W \in \mathbb{R}+^{m\times r}$ y $H \in \mathbb{R}+^{r\times n}$ con la restricción de que todo sea positivo ($\in \mathbb{R}+$). Esto se hace de forma que la aproximación

$$V \approx W \times H$$

tenga un error mínimo.

Las filas de $H$ son los _componentes base_ que se utilizan para reconstruir la matriz original., y se soluciona planteándolo como un problema de optimización matemática con restricciones.

$$\begin{split}
argmin_{W,H}\;& ||V-W\times H||\\
s.t.&\;W,H \in \mathbb{R}_+
\end{split}$$

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/nmf.png" width="40%">

### Obtenemos la descomposición

In [ ]:
# Importar la función NMF del módulo sklearn.decomposition
from sklearn.decomposition import NMF
# Cargar los datos de la base de datos mnist
X = mnist[:,1:785]; y = mnist[:,0]

#Definir un modelo NMF con 15 componentes y una inicialización aleatoria
nmf = NMF(n_components=15, init="random")

#Ajustar el modelo y transformar los datos
Xn = nmf.fit_transform(X)

In [ ]:
# Definir el número de columnas para la figura
cols = 20
#Crear una figura de tamaño 15x3
plt.figure(figsize=(15,3))

#Recorrer cada componente del modelo NMF
for i in range(len(nmf.components_)):
    plt.subplot(len(nmf.components_)/15,15,i+1)# Crear un subplot para cada componente
    # Mostrar el componente como una imagen de escala de grises
    plt.imshow(np.abs(nmf.components_[i].reshape(28,28)), cmap = plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

In [ ]:
# Mostrar la primera fila de los datos transformados
Xn[0,:]

### Verificamos la reconstrucción

In [ ]:
# Crear una figura de tamaño 10x6
plt.figure(figsize=(10,6))
for i in range(6):
    plt.subplot(3,6,i+1)
    # Escoger un índice de ejemplo aleatorio
    k = np.random.randint(len(X))
    plt.imshow(np.abs(np.sum((nmf.components_*Xn[k].reshape(-1,1)), axis=0)).reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])
    plt.subplot(3,6,6+i+1)
    plt.imshow(X[k].reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

### Clasificamos en el nuevo espacio de representación

In [ ]:
# Mostrar la media de la puntuación de la validación cruzada con un modelo GaussianNB en los datos originales
print(np.mean(cross_val_score(GaussianNB(), X,y, cv=5)))
# Mostrar la media de la puntuación de la validación cruzada con un modelo GaussianNB en los datos transformados
print(np.mean(cross_val_score(GaussianNB(), Xn,y, cv=5)))

#### La primera imagen en el nuevo espacio de representación
Observa que todos los componentes son positivos

In [ ]:
# Mostrar la primera fila de los datos transformados
Xn[0]

### Ejemplo: NMF para el reconocimiento de rostros

El código carga un conjunto de datos de imágenes de caras, muestra algunas de ellas en una figura y luego aplica Non-negative Matrix Factorization (NMF) para reducir la dimensionalidad de las imágenes y extraer características relevantes.

In [ ]:
# Se importa la biblioteca pickle para cargar datos almacenados en formato pickle
import pickle
faces = pickle.load(open("data/faces.pkl", "rb"), encoding='latin1')
faces.shape

In [ ]:
# Se crea una figura con una dimension de 15x2
plt.figure(figsize=(15,2))
for i in range(30):
    plt.subplot(2,15,i+1)
    plt.imshow(faces[np.random.randint(len(faces))].reshape(19,19), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

Selección aleatoria de 30 caras de tamaño 19x19. Se utiliza la biblioteca scikit-learn para aplicar NMF en dos versiones diferentes (con diferentes opciones de inicialización) y visualiza las características extraídas en dos figuras diferentes. 

In [ ]:
# Se crea un objeto NMF con 30 componentes y se inicializa aleatoriamente
nmf      = NMF(n_components=30, init="random")
# Se aplica la reducción de dimensiones al conjunto de datos 'faces'
faces_n  = nmf.fit_transform(faces)

In [ ]:
# Resultados
cols=20
plt.figure(figsize=(15,2))
# Se imprime la suma de las componentes de NMF
print(np.sum(nmf.components_))
for i in range(len(nmf.components_)):
    plt.subplot(np.ceil(len(nmf.components_)/15.),15,i+1)
    plt.imshow(np.abs(nmf.components_[i].reshape(19,19)), cmap = plt.cm.Greys)
    plt.xticks([]); plt.yticks([])

La ecuación que se muestra a continuación es una forma de la factorización de matrices no negativas (NMF) con una restricción de dispersión en los componentes.
* La NMF es una técnica de reducción de dimensiones en la que se descompone una matriz de datos en dos matrices más pequeñas, una matriz de componentes base y una matriz de pesos, que combinados pueden aproximarse a la matriz original.
* En esta formulación, se utiliza la norma $L_1$ en los componentes base para forzar la dispersión, lo que significa que los componentes base tendrán valores pequeños y dispersos en lugar de valores grandes y densamente distribuidos.
* La restricción $W,H \in \mathbb{R}_+$ significa que todas las entradas de las matrices $W$ y $H$ deben ser no negativas.

Forzamos dispersión en los componentes, y extendemos el problema de optimización con la norma $L_1$ en los componentes base.

$$\begin{split}
argmin_{W,H}\;& ||V-W\times H|| + ||H||^2_1\\
s.t.&\;W,H \in \mathbb{R}_+
\end{split}$$

Una forma alternativa de la NMF que fuerza la dispersión en la nueva representación, utilizando la norma $L_1$ en la matriz $W$
$$\begin{split}
argmin_{W,H}\;& ||V-W\times H|| + ||W||^2_1\\
s.t.&\;W,H \in \mathbb{R}_+
\end{split}$$


In [ ]:
# Se crea un objeto NMF con 30 componentes, inicializadas por una descomposición en valores singulares no negativos y con una relación L1 de 1
nmf      = NMF(n_components=30, init="nndsvd", l1_ratio=1)
# Se aplica la reducción de dimensiones al conjunto de datos 'faces'
faces_n  = nmf.fit_transform(faces)

In [ ]:
# Resultados
cols=20
plt.figure(figsize=(15,2))
# Se imprime la suma de las componentes de NMF
print(np.sum(nmf.components_))
for i in range(len(nmf.components_)):
    plt.subplot(np.ceil(len(nmf.components_)/15.),15,i+1)
    plt.imshow(np.abs(nmf.components_[i].reshape(19,19)), cmap = plt.cm.Greys)
    plt.xticks([]); plt.yticks([])

___
¡Todo bien! ¡Es todo por hoy! 😀

___

## Taller

### Ejercicio 1

* Cargamos el conjunto de datos de movimientos en la bolsa que está en la ubicación `data/company-stock-movements-2010-2015-incl.csv.gz`
* Convertimos todos los valores en 1 si > 0 y -1 en otro caso
* Aplicamos PCA con 2 componentes al conjunto de datos recién modificado (con +1/-1)
* Usa KMeans con 7 clusters
* Visualiza los clusters de KMeans en el plano 2D dado por PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_clusters = 7

X = PCA ... <YOUR CODE HERE>
y = KMeans ... <YOUR CODE HERE>

Verifica tu código. Debe de aparecer aproximadamente como esto:

In [ ]:
from IPython.display import Image
Image("imgs/companies_clustering.png")

**Código de ayuda para imprimir un texto al lado de cada punto en el plano 2D**:

In [ ]:
cmap = plt.cm.hot
plt.figure(figsize=(20,10))
plt.scatter(X[:,0], X[:,1], color=cmap((y*255./(n_clusters-1)).astype(int)), s=100, edgecolor="black", lw=2)
for i in range(len(d)):
    name = d.index[i]
    plt.text(X[i,0]+.1, X[i,1]+.1, d.index[i], fontsize=14)
plt.axis("off");

### EJercicio 3: Bag of features
Construiremos en este conjunto de problemas una implementación del método de _Bag of features_, sobre el dataset MNIST.

In [ ]:
Image(filename='imgs/bof.png')

Ejecuta las siguientes celdas para cargar MNIST y ver una muestra del mismo.

In [ ]:
mnist = pd.read_csv("data/mnist1.5k.csv.gz", compression="gzip", header=None).values.astype(float)
print("Dimensión de los datos originales", mnist.shape)
X=mnist[:,1:785]
y=mnist[:,0]

In [ ]:
perm = np.random.permutation(range(X.shape[0]))[0:50]
random_imgs   = X[perm]
random_labels = y[perm] 
fig = plt.figure(figsize=(10,6))
for i in range(random_imgs.shape[0]):
    ax=fig.add_subplot(5,10,i+1)
    plt.imshow(random_imgs[i].reshape(28,28), interpolation="nearest", cmap = plt.cm.Greys_r)
    ax.set_title(int(random_labels[i]))
    ax.set_xticklabels([])
    ax.set_yticklabels([])

### Ejercicio 4: Extracción de parches

Completa la siguiente función para que dada una imagen en escala de grises ($\in [0,255]^{h\times w}$), extraiga parches de tamaño cuadrado de la misma, con un tamaño de paso concreto, tendiendo en cuenta que:

- los parches cuya suma de valores sean cero sólo deberán de ser incluidos sin `include_empty_patches` es verdadero.
- sólo se incluirán parches completos (es decir, de tamaño `patch_size` $\times$ `patch_size`)

Por ejemplo, para la siguiente imagen:

           img= [[5 8 2 4 1 4 6 8 0 6 1]
                 [3 8 1 4 3 5 6 9 3 1 7]
                 [8 7 3 4 2 7 6 0 9 3 8]
                 [4 7 3 2 7 2 4 7 5 0 5]
                 [9 8 7 6 5 1 8 7 0 6 4]
                 [0 3 8 4 7 0 0 3 5 5 2]
                 [5 4 3 2 1 0 0 8 8 2 8]
                 [8 3 7 8 2 5 0 3 8 2 4]
                 [6 7 2 8 0 0 1 7 5 4 8]]
         
la ejecución de `extract_patches(img, 2,5)` da tres parches (el parche con cuatro 0's se excluye por `include_empty_patches=False`):

        [[5 8]   [[4 6]   [[0 3]
         [3 8]]   [5 6]]   [5 4]]

In [ ]:
def extract_patches(img, patch_size, step_size, include_empty_patches=False):
    patches = []
    for y in range(0, img.shape[0]-patch_size+1, step_size):
        for x in ...:
            patch = ...
    return patches

import urllib, inspect
src1 = urllib.quote_plus(inspect.getsource(extract_patches))

Comprueba tu código

In [ ]:
img = np.array([[5, 8, 2, 4, 1, 4, 6, 8, 0, 6, 1],
       [3, 8, 1, 4, 3, 5, 6, 9, 3, 1, 7],
       [8, 7, 3, 4, 2, 7, 6, 0, 9, 3, 8],
       [4, 7, 3, 2, 7, 2, 4, 7, 5, 0, 5],
       [9, 8, 7, 6, 5, 1, 8, 7, 0, 6, 4],
       [0, 3, 8, 4, 7, 0, 0, 3, 5, 5, 2],
       [5, 4, 3, 2, 1, 0, 0, 8, 8, 2, 8],
       [8, 3, 7, 8, 2, 5, 0, 3, 8, 2, 4],
       [6, 7, 2, 8, 0, 0, 1, 7, 5, 4, 8]])
for i in extract_patches(img, 2,5):
    print(i)

Observa los parches extraídos de una imagen

In [ ]:
patch_size, step_size = 7, 2

p = extract_patches(X[1307].reshape(28,28), patch_size, step_size, include_empty_patches=True)
print "patch extraction with empty patches", len(p)
pe = extract_patches(X[1307].reshape(28,28), patch_size, step_size)
print "without empty patches", len(pe)
plt.figure(figsize=(5,5))
s = np.sqrt(len(p))
for i in range(len(p)):
    plt.subplot(s,s,i+1)
    plt.imshow(p[i], cmap = plt.cm.Greys_r, interpolation="nearest")
    plt.xticks([]); plt.yticks([])

### Ejercicio 5: Diccionario visual

Fíjate cómo funciona KMeans. Trata de entender el código y ejecútalo varias veces

In [ ]:
from sklearn.datasets import make_blobs
Xp, _ = make_blobs(n_samples=200,n_features=2, centers=4, cluster_std=2)
cols = ["red", "blue", "green", "gray"]

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.scatter(Xp[:,0], Xp[:,1], color="gray")
plt.xticks([]); plt.yticks([])
plt.subplot(122)
plt.scatter(Xp[:,0], Xp[:,1])
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(Xp)
y = km.predict(Xp)
for i in np.unique(y):
    plt.scatter(Xp[y==i][:,0], Xp[y==i][:,1], color=cols[i])
    plt.xticks([]); plt.yticks([]);
for i,p in enumerate(km.cluster_centers_):
    plt.scatter(p[0], p[1], s=100, c="black", marker="x", lw=5)
    plt.xticks([]); plt.yticks([]);

Vamos a hacer un KMeans con los parches extraídos de las imágenes y consideraremos los centroides como palabras visuales. Para ello, completa la función siguiente de forma que:

- tendrás que usar la función `extract_patches` del ejercicio anterior.
- construye una lista con todos parches de las imágenes de la matriz `X`. Observa que en cada file de la matriz hay una imagen linearizada y tendrás que hacer un `.reshape(28,28)` para convertirla en una matriz antes de llamar a `extract_patches`.
- devuelve la lista de centroides del `KMeans`. Consulta la documentación de [KMeans en sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

Esta lista de centroides será nuestro **diccionario visual** y, como cada centroide tiene el mismo tamaño que los parches, los podremos visualizar. A continuación, completa el código de tal forma que se pueda visualizar el diccionario de palabras.

In [ ]:
def get_visual_dictionary(X, patch_size, step_size, dict_size):
    from sklearn.cluster import KMeans
    patches = []
    for img in X:
        patches_in_this_image = ...
        patches.append(...)

    cinit = np.zeros((dict_size, patch_size**2))
    km = KMeans(n_clusters=..., init=cinit, n_init=1)#, n_jobs=24)
    km.fit(...)
    return ...

import urllib, inspect
src2 = urllib.parse.quote_plus(inspect.getsource(get_visual_dictionary))

Comprueba tu código. la siguiente configuración te debería de dar un conjunto de palabras visuales como este:

In [ ]:
Image(filename='imgs/vwords.png')

In [ ]:
patch_size, step_size, dict_size = 7, 7, 30
vwords = get_visual_dictionary(X[:100], patch_size, step_size, dict_size)
plt.figure(figsize=(15,1.5))
vwords = vwords[np.argsort([np.sum(vwords[i]) for i in range(len(vwords))])]
print(np.sum([np.sum(vwords[i]) for i in range(len(vwords))]))
n = len(vwords)
for i in range(n):
    plt.subplot(1,n,i+1)
    plt.imshow(vwords[i].reshape(patch_size,patch_size),cmap = plt.cm.Greys_r,interpolation="nearest")
    plt.xticks([]); plt.yticks([])

### Ejercicio 6: Histograma de palabras visuales

Vamos ahora a calcular de qué palabras visuales consta cada imagen. Esta será la representación _Bag of Features_ ya que, para cada palabra visual, contaremos cuantas veces aparece en cada imagen. Esto es análogo a contar, en un documento, cuantas veces aparece cada palabra.

Para ello haremos dos funciones:

- `get_closest`: en la que dado un parche y un diccionario visual como el devuelto en el ejercicio anterior nos de la palabara visual más parecida. Esta similitud estará medida en términos de la norma L2 (`np.linalg.norm`)

- `get_histogram`:  que, dada una imagen y un diccionario, (1) extrae todos los parches de la imagen, (2) para cada parche obtiene cual es su palabra visual más similar y (3) devuelve un vector con tantos elementos como palabras visuales, con la frecuencia relativa de aparición de cada palabra visual en la imagen. La frecuencia relativa de una palabra visual $w$ viene dada por el número de parches de la imagen cuya palabra visual más similar es $w$, dividido por el número total de parches.

Completa el código de ambas funciones

In [ ]:
def get_closest(patch, dictionary):
    r = ...
    return r

def get_histogram(img, patch_size, step_size, dictionary):

    histogram = ...
    return histogram

import urllib, inspect
src3 = urllib.parse.quote_plus(inspect.getsource(get_closest)+inspect.getsource(get_histogram))

Comprueba tu código. La celda siguiente selecciona un parche al azar de una imagen al azar y muestra su palabra visual más cercana.

In [ ]:
patch_size, step_size = 7, 2

mnist = pd.read_csv("data/mnist1.5k.csv.gz", compression="gzip", header=None).values.astype(float)
X=mnist[:,1:785]
y=mnist[:,0]
img = X[np.random.randint(len(X))].reshape(28,28)
patches = extract_patches(img, patch_size, step_size)
patch   = patches[np.random.randint(len(patches))].reshape(patch_size**2)
vword = vwords[get_closest(patch, vwords)]

plt.figure(figsize=(6,3))
plt.subplot(121)
plt.imshow(patch.reshape(patch_size, patch_size),  cmap = plt.cm.Greys_r, interpolation="nearest")
plt.title("patch")
plt.xticks([]); plt.yticks([])
plt.subplot(122)
plt.title("visual word")
plt.imshow(vword.reshape(patch_size, patch_size),  cmap = plt.cm.Greys_r, interpolation="nearest")
plt.xticks([]); plt.yticks([]);

La siguiente celda selecciona una imagen al azar y obtiene su histograma de palabras visuales.

In [ ]:
k = np.random.randint(len(X))
print k
img = X[k].reshape(28,28)
h = get_histogram(img, patch_size, step_size, vwords)
plt.imshow(img, cmap = plt.cm.Greys_r, interpolation="nearest")

plt.figure(figsize=(17,3))
n = len(vwords)
for i in range(n):
    plt.subplot(1,n,i+1)
    plt.imshow(vwords[i].reshape(patch_size,patch_size),cmap = plt.cm.Greys_r)
    plt.title("%.2f"%h[i])
    plt.xticks([]); plt.yticks([])

#### Experimenta con la nueva representación

Observa cómo aumenta el porcentaje de acierto representando las imágenes con un histograma de palabras visuales. Este proceso puede demorarse varios minutos.

In [ ]:
patch_size, step_size = 7,2
#vdict = get_visual_dictionary(X, patch_size, step_size, 60)
Xh = []
for i, img in enumerate(X):
    print("\r",i,)
    Xh.append(get_histogram(img.reshape(28,28), patch_size, step_size, vdict))
    
Xh = np.array(Xh)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

estimator = GaussianNB()
sc = cross_val_score(estimator, X, y, cv=5)
print("Original pixels                %.3f +/- %.3f"%(np.mean(sc), np.std(sc)))
sc = cross_val_score(estimator, Xh, y, cv=5)
print("Bag of features representation %.3f +/- %.3f"%(np.mean(sc), np.std(sc)))